# graph-tool的入门使用

In [48]:
from graph_tool.all import *

In [49]:
# 创建一个空graph
g = Graph()

In [50]:
# 默认是directed的
g = Graph(directed=False)

In [51]:
# 添加结点
v1 = g.add_vertex()
v2 = g.add_vertex()
v3 = g.add_vertex()
v4 = g.add_vertex()
v5 = g.add_vertex()

In [52]:
# 添加边
e1 = g.add_edge(v1, v2)
e2 = g.add_edge(v1, v3)
e3 = g.add_edge(v2, v3)
e4 = g.add_edge(v3, v4)
e5 = g.add_edge(v4, v5)
e6 = g.add_edge(v3, v5)

In [53]:
# 图的可视化
graph_draw(g, vertex_text=g.vertex_index, output="demo1.png")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x17188e280, at 0x10e0fe400>

<div>
<img src='demo1.png' width='300' height='300'/>
</div>

In [54]:
# 查看某个结点的出度
print(v1.out_degree())

2


In [55]:
# 查看边的特征
print(e1.source(), e1.target())

0 1


In [56]:
# 每个结点在图中有固定的下标，可以通过vertex_index获取
print(g.vertex_index[v1])

0


In [57]:
# 删除节点和边
g.remove_edge(e1)  
g.remove_vertex(v1)  

In [58]:
graph_draw(g, vertex_text=g.vertex_index, output="demo2.png")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x17188e280, at 0x171895130>

<div>
<img src='demo2.png' width='200' height='200'/>
</div>

In [59]:
# 通过下标获取结点和边
v = g.vertex(0)
print(v)
e = g.edge(0, 1)
print(e)

0
(0, 1)


In [60]:
# 迭代查看全部结点和边
for v in g.vertices():
    print(v)
print('-----------')
for e in g.edges():
    print(e)

0
1
2
3
-----------
(0, 1)
(1, 2)
(1, 3)
(2, 3)


In [61]:
# 迭代查看全部节点的邻居节点
for v in g.vertices():
#    for e in v.out_edges():
#        print(e)
   print('v: ', v)
   for w in v.out_neighbors():
       print(w)
   print('------')
    

v:  0
1
------
v:  1
2
3
0
------
v:  2
3
1
------
v:  3
2
1
------


In [62]:
# 迭代查看全部节点的邻居节点的边
for v in g.vertices():
   print('v: ', v)
   for e in v.out_edges():
       print(e)
   print('------')

v:  0
(0, 1)
------
v:  1
(1, 2)
(1, 3)
(1, 0)
------
v:  2
(2, 3)
(2, 1)
------
v:  3
(3, 2)
(3, 1)
------


# 属性映射
属性映射是一种将附加信息与顶点、边或图形本身关联的方法。

因此有三种类型的属性映射：顶点、边和图。它们由类VertexPropertyMap, EdgePropertyMap, and GraphPropertyMap处理。每个创建的特性映射都有一个关联的值类型，必须从预定义的集合中选择：

---

<div>
<img src='imgs/pro_map.png' width='400' height='400'/>
</div>

## 使用Property maps 给图、结点、边 赋固定type的值

 使用： new_vertex_property()，new_edge_property() ，new_graph_property() 

In [88]:
from numpy.random import randint

g1 = Graph()
g1.add_vertex(10) # 添加10个结点

# insert some random links
for s,t in zip(randint(0, 10,10), randint(0, 10,10)):  # 任意生成两个0-10之间整数
    g1.add_edge(g1.vertex(s), g1.vertex(t))

vprop_double = g1.new_vertex_property("double")            # Double类型的结点
v = g1.vertex(5)
vprop_double[v] = 3.1416

vprop_vint = g1.new_vertex_property("vector<int>")         # ints类型的结点
v = g1.vertex(6)
vprop_vint[v] = [1, 3, 6]

eprop_dict = g1.new_edge_property("object")                # python中任意类型的边
e = g1.edges().next()
eprop_dict[e] = {"foo": "bar", "gnu": 42}                 # In this case, a dict.

gprop_bool = g1.new_graph_property("bool")                 # Boolean类型的graph
gprop_bool[g1] = True

In [99]:
vprop_vint[6]

array([1, 3, 6], dtype=int32)

In [90]:
graph_draw(g1, vertex_text=g1.vertex_index, output="demo3.png")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1718955b0, at 0x17188bb20>

<div>
<img src='demo3.png' width='500' height='500'/>
</div>

## 内部属性映射

可以将任何创建的属性映射设置为对应图的自身属性。 这意味着它将被复制并与图形一起保存到文件中。

通过将属性包含在图形的字典式属性中来对属性进行内部化vertex_properties，edge_properties或graph_properties。当插入到图中时，属性映射必须具有唯一的名称（在相同类型的名称之间）：


In [93]:
from graph_tool.all import *

# 定义图
g2 = Graph()

# 定义多种节点特征
vprop_name = g2.new_vertex_property("string")
vprop_concepts = g2.new_vertex_property("vector<string>")
vprop_sentidxs = g2.new_vertex_property("vector<int32_t>")
# 添加节点
v = g2.add_vertex()
# 给结点添加数据
vprop_name[v] = 'apple'
vprop_concepts[v] = ['A', 'B']
vprop_sentidxs[v] = [1,2]


# 任何创建的property可以设定为某个graph的自身属性，这样就意味着，当你copy和存储图时，这些属性会更图一起存在。

# 把这些属性插入到图中，同时取一个名字
g2.vertex_properties["name"] = vprop_name
g2.vertex_properties["relations"] = vprop_concepts
g2.vertex_properties["ids"] = vprop_sentidxs

In [94]:
graph_draw(g_1, vertex_text=g_1.vertex_index, output="demo4.png")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x16f3498b0, at 0x1718ac640>

<div>
<img src='demo4.png' width='200' height='200'/>
</div>

In [111]:
g2.list_properties()

ids            (vertex)  (type: vector<int32_t>)
name           (vertex)  (type: string)
relations      (vertex)  (type: vector<string>)


In [112]:
g1.list_properties()

In [113]:
g.list_properties()